In [1]:
import json, time, os
from datetime import datetime
import pandas as pd


from Montecarlo import montecarlo_dias, montecarlo_base, montecarlo_viento, montecarlo_cierre
from TraficoAEP import TraficoAviones
from TraficoAEPCierre import TraficoAEPCerrado, AEPCerrado
from TraficoAEPViento import TraficoAEPViento  


OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

def save_result(df: pd.DataFrame, meta: dict, name: str) -> str:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    base = f"{ts}_{name}"
    csv_path = os.path.join(OUTDIR, base + ".csv")
    json_path = os.path.join(OUTDIR, base + ".meta.json")
    df.to_csv(csv_path, index=False)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("guardado:", csv_path)
    return csv_path


In [2]:
lams = [0.08, 0.10, 0.12, 0.14]

In [3]:

df_base = montecarlo_base(lams=lams, dias=200, seed=123)
save_result(df_base, {"scenario":"base", "lams":lams, "dias":200, "seed":123}, "base_mc")
df_base.head()


guardado: outputs\20250920-172525_base_mc.csv


,lam,congestion_rate_mean,congestion_rate_ci_low,congestion_rate_ci_high,avg_delay_min_mean,avg_delay_min_ci_low,avg_delay_min_ci_high,divert_rate_mean,divert_rate_ci_low,divert_rate_ci_high,avg_arrivals_per_day,avg_diverted_per_day,dias
0,0.08,0.075694,0.072875,0.078513,1.042879,0.970139,1.115618,0.130541,0.125910,0.135172,86.475,11.375,200
1,0.10,0.089529,0.087066,0.091992,1.549081,1.461334,1.636828,0.161046,0.156620,0.165473,108.565,17.630,200
2,0.12,0.107303,0.104734,0.109873,2.227680,2.124110,2.331250,0.192186,0.187901,0.196471,129.515,25.045,200
3,0.14,0.118811,0.116294,0.121328,2.922179,2.775731,3.068627,0.224651,0.220946,0.228357,151.330,34.155,200


In [4]:
kw = {"p_goaround": 0.10, "final_threshold_nm": 3}
df_wind = montecarlo_dias(
    lams=lams, dias = 200, seed=123,
    Controller=TraficoAEPViento, controller_kwargs=kw
)
df_wind["scenario"] = "viento"
save_result(df_wind, {"scenario":"viento", "kwargs":kw, "lams":lams, "dias":200, "seed":123}, "viento_mc")
df_wind.head()

guardado: outputs\20250920-172634_viento_mc.csv


,lam,congestion_rate_mean,congestion_rate_ci_low,congestion_rate_ci_high,avg_delay_min_mean,avg_delay_min_ci_low,avg_delay_min_ci_high,divert_rate_mean,divert_rate_ci_low,divert_rate_ci_high,avg_arrivals_per_day,avg_diverted_per_day,dias,scenario
0,0.08,0.045672,0.043022,0.048323,1.957539,1.846191,2.068887,0.159317,0.154292,0.164342,86.475,13.900,200,viento
1,0.10,0.051625,0.049207,0.054043,2.651926,2.514382,2.789470,0.201357,0.196697,0.206016,108.565,22.045,200,viento
2,0.12,0.060232,0.057762,0.062701,3.208975,3.044476,3.373475,0.243566,0.239085,0.248047,129.515,31.725,200,viento
3,0.14,0.065280,0.062944,0.067616,3.978418,3.787440,4.169396,0.281609,0.277684,0.285533,151.330,42.820,200,viento


In [7]:
import importlib
import Simulacion, TraficoAEPCierre, Montecarlo

importlib.reload(Simulacion)
importlib.reload(TraficoAEPCierre)
importlib.reload(Montecarlo)

closure = AEPCerrado(start_min=180, dur_min=30)
kwargs_cierre = {"closure": closure}

meta = {
    "scenario": "cierre",
    "kwargs": {
        "closure": {"start_min": closure.start_min, "dur_min": closure.dur_min}
    },
    "lams": lams, "dias": 200, "seed": 123
}

df_close = montecarlo_dias(
    lams=lams, dias=200, seed=123,
    Controller=TraficoAEPCerrado, controller_kwargs=kwargs_cierre
)

df_close["scenario"] = "cierre"
save_result(df_close, meta, "cierre_mc")
df_close.head()


guardado: outputs\20250920-174008_cierre_mc.csv


,lam,congestion_rate_mean,congestion_rate_ci_low,congestion_rate_ci_high,avg_delay_min_mean,avg_delay_min_ci_low,avg_delay_min_ci_high,divert_rate_mean,divert_rate_ci_low,divert_rate_ci_high,avg_arrivals_per_day,avg_diverted_per_day,dias,scenario
0,0.08,0.076869,0.074080,0.079659,1.859125,1.729921,1.988330,0.131109,0.126465,0.135753,86.475,11.425,200,cierre
1,0.10,0.090892,0.088454,0.093330,2.415214,2.288325,2.542104,0.162033,0.157642,0.166425,108.565,17.740,200,cierre
2,0.12,0.108400,0.105836,0.110965,3.205592,3.059515,3.351669,0.194762,0.190519,0.199004,129.515,25.380,200,cierre
3,0.14,0.120044,0.117543,0.122545,4.006589,3.835971,4.177207,0.228795,0.225024,0.232567,151.330,34.790,200,cierre
